In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import os
from datetime import datetime, timedelta
!pip install meteostat
from meteostat import Hourly, Stations
import holidays
import requests
import json
import folium
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

def read_dataframe_from_parquet(filename):
    return pd.read_parquet(filename)



all_df = []
for file in tqdm(os.listdir("/content/drive/MyDrive/MIT805")):
    if file.endswith(".parquet"):
        df = read_dataframe_from_parquet("/content/drive/MyDrive/MIT805/" + file)
        all_df.append(df)

df = pd.concat(all_df, ignore_index=True)
del all_df
print("Combined DataFrame shape:", df.shape)
if df.empty:
    print("No data to visualize.")

df['bikes_in_electric_bike'] = df['bikes_in_electric_bike'].fillna(0)
df['bikes_in_classic_bike'] = df['bikes_in_classic_bike'].fillna(0)
df['bikes_out_classic_bike'] = df['bikes_out_classic_bike'].fillna(0)
df['bikes_out_electric_bike'] = df['bikes_out_electric_bike'].fillna(0)
df = df[df['date'] >= datetime(2020,1,1)] # some data in the files are tests prior to this date

In [3]:


start = datetime(2020, 1, 1, 0, 0)
end = datetime(2025, 8, 31, 23, 59)
stations = Stations()

weather_by_hour = Hourly('74486', start, end).fetch() # JFK Airport. Only NYC Stations with precip info
# print(weather_by_hour.head())
ny_holidays = holidays.US(state="NY")
weather_by_hour["prcp"] = weather_by_hour["prcp"].fillna(0)

In [ ]:
join_df = df.copy().join(weather_by_hour, on="date", how="left")

# print(df.head())
join_df["date_only"] = df["date"].dt.date
sub_df = (
    join_df.groupby(by=["date_only"])
    .agg({"total_trips": "sum", "is_holiday": "first", "is_weekend": "first"})
    .reset_index()
)
# print(sub_df.tail(150))
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=sub_df,
    x="date_only",
    y="total_trips",
    palette="tab10",
)
for _, per in sub_df[sub_df['is_holiday'] == 1].iterrows():
  plt.axvspan(per['date_only'], (per['date_only'] + timedelta(days=1)), facecolor='red', alpha=0.5)
for _, per in sub_df[sub_df['is_weekend'] == 1].iterrows():
  plt.axvspan(per['date_only'], (per['date_only'] + timedelta(days=1)), facecolor='orange', alpha=0.5)
plt.title("Total Trips Over Time")
plt.xlabel("Date")
plt.ylabel("Total Trips")
plt.show()
plt.close()



In [ ]:
join_df1 = join_df.copy()
join_df1 = (join_df1.groupby(by=["date"])
              .agg({"total_trips": "sum", "is_holiday": "first", "is_weekend": "first", "temp": "first", "prcp": "first"})
              .reset_index())

join_df1["date"] = join_df1["date"].dt.date
join_df1 = (join_df1.groupby(by=["date"])
              .agg({"total_trips": "sum", "is_holiday": "first", "is_weekend": "first", "temp": "mean", "prcp": "sum"})
              .reset_index())
join_df2 = join_df1.copy()

# print(join_df1.head())
def num_features(the_df):
  new_df = the_df.copy()
  new_df["date"] = pd.to_datetime(new_df["date"])
  new_df["year"] = new_df["date"].dt.year
  new_df["month"] = new_df["date"].dt.month
  new_df["day"] = new_df["date"].dt.day

  new_df["day_of_week"] = new_df["date"].dt.dayofweek
  new_df["season"] = new_df["month"].apply(lambda x: 1 if x in [12, 1, 2] else 2 if x in [3, 4, 5] else 3 if x in [6, 7, 8] else 4)
  new_df = new_df.drop(columns=["date"])
  return new_df

split_date = datetime(2024, 9, 1).date()
train_df = num_features(join_df1.loc[join_df1["date"] <= split_date].copy())
train_X = train_df.drop(columns=["total_trips"])
train_Y = train_df["total_trips"]
test_df = num_features(join_df1.loc[join_df1["date"] > split_date].copy())
test_X = test_df.drop(columns=["total_trips"])
test_Y = test_df["total_trips"]

reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(train_X, train_Y,
        eval_set=[(train_X, train_Y), (test_X, test_Y)],
       verbose=False)

# _ = plot_importance(reg, height=0.9)

test_Y_xgb = reg.predict(test_X)



def onehot_features(the_df):
  new_df = num_features(the_df)

  # one hot for lin reg
  new_df = pd.get_dummies(new_df, columns=['month', 'season', 'day_of_week', 'day'], drop_first=True)

  return new_df

lin_test_df_full = join_df2.loc[join_df2["date"] <= split_date].copy()
lin_train_df = onehot_features(lin_test_df_full)
lin_train_X = lin_train_df.drop(columns=["total_trips"])
lin_train_Y = lin_train_df["total_trips"]
lin_test_df_full = join_df2.loc[join_df2["date"] > split_date].copy()
lin_test_df = onehot_features(lin_test_df_full)
lin_test_X = lin_test_df.drop(columns=["total_trips"])
lin_test_Y = lin_test_df["total_trips"]

scaler = StandardScaler()
lin_X_train_scaled = scaler.fit_transform(lin_train_X)
lin_X_test_scaled = scaler.transform(lin_test_X)


lasso_model = Lasso(alpha=0.2)
lasso_model.fit(lin_X_train_scaled, train_Y)

lin_y_pred = lasso_model.predict(lin_X_test_scaled)
# print(list(zip(test_X.columns, lasso_model.coef_)))

plt.figure(figsize=(12, 6))
feat_df = pd.DataFrame({'Feature': lin_test_X.columns, 'Coeffiecient': np.abs(lasso_model.coef_), "Correlation": np.sign(lasso_model.coef_) })
feat_df = feat_df.sort_values(by='Coeffiecient', ascending=False)
sns.barplot(x='Coeffiecient', y='Feature', data=feat_df.head(10), hue="Correlation", palette=sns.diverging_palette(0, 120, l=35, center="light", as_cmap=True))
plt.title('Top 10 Lasso Linear Regression Coefficients')
plt.xlabel('Feature')
plt.ylabel('Coefficient')
plt.xticks(rotation=45)
plt.show()



In [ ]:
plt.figure(figsize=(12, 6))

test_Xd = test_X
test_Xd["date"] = lin_test_df_full["date"]
test_Xd = test_Xd[["date"]]
test_Xd = pd.to_datetime(test_Xd["date"])
# print(test_Xd
plt.plot(test_Xd, test_Y, label='Actual')
plt.plot(test_Xd, lin_y_pred, label='Predicted')


plt.title('Linear Regression with Lasso: Actual vs Predicted Total Trips')
plt.xlabel('Date')
plt.ylabel('Total Trips')
plt.legend()
plt.show()

y_true, y_pred = np.array(test_Y), np.array(lin_y_pred)
print(np.mean(np.abs((y_true - y_pred) / y_true)) * 100)

In [ ]:
plt.figure(figsize=(12, 6))

test_Xd = test_X
test_Xd["date"] = test_Xd["year"].astype(str) + "-" + test_Xd["month"].astype(str) + "-" + test_Xd["day"].astype(str)
test_Xd = pd.to_datetime(test_Xd["date"])
plt.plot(test_Xd, test_Y, label='Actual')
plt.plot(test_Xd, test_Y_xgb, label='Predicted')


plt.title('XGBoost: Actual vs Predicted Total Trips')
plt.xlabel('Date')
plt.ylabel('Total Trips')
plt.legend()
plt.show()

y_true, test_Y_xgb = np.array(test_Y), np.array(test_Y_xgb)
print(np.mean(np.abs((y_true - test_Y_xgb) / y_true)) * 100)

In [8]:
weather_by_day = weather_by_hour.copy()
weather_by_day["date"] = pd.to_datetime(weather_by_day.index)
weather_by_day["date"] = pd.to_datetime(weather_by_day["date"].dt.date)
weather_by_day = weather_by_day.groupby(by=["date"]).agg({"temp": "mean", "prcp": "sum"}).reset_index()
weather_by_day["date"] = pd.to_datetime(weather_by_day["date"])



In [ ]:
forecast_dates = pd.date_range(start='2025-09-01', periods=365, freq='D')


forecast_df = pd.DataFrame({'date': forecast_dates})

forecast_df['year'] = forecast_df['date'].dt.year
forecast_df['month'] = forecast_df['date'].dt.month
forecast_df['day'] = forecast_df['date'].dt.day
forecast_df['day_of_week'] = forecast_df['date'].dt.dayofweek
forecast_df['season'] = forecast_df['month'].apply(lambda x: 1 if x in [12, 1, 2] else 2 if x in [3, 4, 5] else 3 if x in [6, 7, 8] else 4)


ny_holidays = holidays.US(state="NY", years=forecast_df['year'].unique())
forecast_df['is_holiday'] = forecast_df['date'].apply(lambda x: 1 if x in ny_holidays else 0)
forecast_df['is_weekend'] = forecast_df['date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)


weather_by_day["month"] = weather_by_day["date"].dt.month
weather_by_day["day"] = weather_by_day["date"].dt.day
weather_by_day_avg = (weather_by_day.groupby(by=["month", "day"])
                      .agg({"temp": "mean", "prcp": "mean"})
                      .reset_index()
                    )
forecast_df = pd.merge(forecast_df, weather_by_day_avg, on=['month', 'day'], how='left')


forecast_X = forecast_df.drop(columns=['date'])


forecast_X = forecast_X[train_X.columns]

forecast_Y_xgb = reg.predict(forecast_X)


confidence_band = 0.20 * forecast_Y_xgb
lower_bound = forecast_Y_xgb - confidence_band
upper_bound = forecast_Y_xgb + confidence_band

plt.figure(figsize=(12, 6))
plt.plot(forecast_dates, forecast_Y_xgb, label='Predicted')
plt.fill_between(forecast_dates, lower_bound, upper_bound, color='blue', alpha=0.2, label='20% Confidence Band')

plt.title('Predicted Total Trips with Confidence Band for Next Year')
plt.xlabel('Date')
plt.ylabel('Total Trips')
plt.legend()
plt.show()

In [ ]:


confidence_band = 0.20 * y_pred
lower_bound = y_pred - confidence_band
upper_bound = y_pred + confidence_band

plt.figure(figsize=(12, 6))
plt.plot(forecast_dates[:-1], y_pred, label='Predicted')
plt.fill_between(forecast_dates[:-1], lower_bound, upper_bound, color='blue', alpha=0.2, label='25% Confidence Band')

plt.title('Predicted Total Trips with Confidence Band for Next Year')
plt.xlabel('Date')
plt.ylabel('Total Trips')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(sub_df['date_only'], sub_df['total_trips'], label='Historical Actual')

plt.plot(forecast_dates[:-1], y_pred, label='Predicted (Linear Regression with Lasso)', color="lightblue")

confidence_band = 0.25 * y_pred
lower_bound = y_pred - confidence_band
upper_bound = y_pred + confidence_band
plt.fill_between(forecast_dates[:-1], lower_bound, upper_bound, color='blue', alpha=0.2, label='25% Confidence Band')


plt.title('Historical and Predicted Total Trips')
plt.xlabel('Date')
plt.ylabel('Total Trips')
plt.legend()
plt.show()

In [ ]:


url = "https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json"
response = requests.get(url)
data = json.loads(response.text)
stations = pd.DataFrame(data['data']['stations'])
print(stations.head())


In [ ]:
start_date = "2025-07-01" # @param {"type":"date"}
end_date = "2025-07-06" # @param {"type":"date"}
threshold = 10.0 # @param {"type":"number"}
threshold = threshold/100

df_agg = df[df["date"] >= start_date][df["date"] < end_date].copy()

df_agg = df_agg.groupby('station').agg({
    'bikes_out_classic_bike': 'sum',
    'bikes_out_electric_bike': 'sum',
    'bikes_in_classic_bike': 'sum',
    'bikes_in_electric_bike': 'sum'
}).reset_index()


df_agg['total_bikes_out'] = df_agg['bikes_out_classic_bike'] + df_agg['bikes_out_electric_bike']
df_agg['total_bikes_in'] = df_agg['bikes_in_classic_bike'] + df_agg['bikes_in_electric_bike']


df_agg['net_flow'] = df_agg['total_bikes_in'] - df_agg['total_bikes_out']

df_agg['total_flow'] = (df_agg['total_bikes_in'] + df_agg['total_bikes_out'])/2





merged_df = pd.merge(df_agg, stations[['short_name', 'lat', 'lon', 'name']], left_on='station', right_on='short_name', how='left')

merged_df.dropna(subset=[ 'lat', 'lon'], inplace=True)


import folium
m = folium.Map(location=[merged_df['lat'].mean(), merged_df['lon'].mean()], zoom_start=12)


for index, row in merged_df.iterrows():
    color = ('green' if row['net_flow'] >= threshold * merged_df['net_flow'].max() else
     "red" if row['net_flow'] <= threshold * merged_df['net_flow'].min() else "grey")

    log_total_flow = np.log(row['total_flow']) / np.log(merged_df['total_flow'].max())
    radius = log_total_flow * 15

    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        tooltip=f"Station: {row['name']}<br>Total Flow: {row['total_flow']}<br>Net Flow: {row['net_flow']}"
    ).add_to(m)

m

In [ ]:
start_date = "2025-06-01" # @param {"type":"date"}
end_date = "2025-07-10" # @param {"type":"date"}
under_utilised_threshold = 50.0 # @param {"type":"number"}
over_utilised_threshold = 100.0 # @param {"type":"number"}


df_agg1 = df[df["date"] >= start_date][df["date"] < end_date].copy()

df_agg1 = df_agg1.groupby(by=['date', 'station'], as_index=True).agg({
    'bikes_out_classic_bike': 'sum',
    'bikes_out_electric_bike': 'sum',
    'bikes_in_classic_bike': 'sum',
    'bikes_in_electric_bike': 'sum',
}).reset_index()


df_agg1['total_bikes_out'] = df_agg1['bikes_out_classic_bike'] + df_agg1['bikes_out_electric_bike']
df_agg1['total_bikes_in'] = df_agg1['bikes_in_classic_bike'] + df_agg1['bikes_in_electric_bike']


df_agg1['net_flow'] = df_agg1['total_bikes_in'] - df_agg1['total_bikes_out']

df_agg1['total_flow'] = (df_agg1['total_bikes_in'] + df_agg1['total_bikes_out'])/2





merged_df1 = pd.merge(df_agg1, stations[['short_name', 'lat', 'lon', 'name', 'capacity']], left_on='station', right_on='short_name', how='left').set_index(["date", "station"])

merged_df1["Utilisation %"] = (merged_df1["total_flow"] / merged_df1["capacity"]) * 100

merged_df1 = merged_df1[merged_df1["capacity"] != 0]
merged_df1.dropna(subset=["capacity"], inplace=True)

merged_df1 = merged_df1.groupby(by=["station"]).agg({
    "lat": "first",
    "lon": "first",
    "capacity": "first",
    "total_flow": "sum",
    "Utilisation %": "max",
    "name": "first"
}).reset_index()


m = folium.Map(location=[merged_df1['lat'].mean(), merged_df1['lon'].mean()], zoom_start=12)

for index, row in merged_df1.iterrows():
    color = ('red' if row['Utilisation %'] >= over_utilised_threshold else
     "blue" if row['Utilisation %'] <= under_utilised_threshold else "black")

    log_total_flow = row['capacity']/merged_df1['capacity'].max()
    radius = log_total_flow * 15

    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        tooltip=f"Station: {row['name']}<br>Total Flow: {row['total_flow']}<br>Utilisation %: {row['Utilisation %']}<br>Capacity: {row['capacity']}"
    ).add_to(m)
m


In [ ]:
start_date = "2020-01-01" # @param {"type":"date"}
end_date = "2025-08-31" # @param {"type":"date"}
df_bike_type = df.copy()
df_bike_type = df_bike_type[df_bike_type["date"] >= start_date][df_bike_type["date"] < end_date]
df_bike_type['date_only'] = df_bike_type['date'].dt.date

df_bike_type_agg = df_bike_type.groupby(['date_only']).agg({
    'bikes_out_classic_bike': 'sum',
    'bikes_out_electric_bike': 'sum',
    'bikes_in_classic_bike': 'sum',
    'bikes_in_electric_bike': 'sum'
}).reset_index()

df_bike_type_agg['total_classic_bikes'] = (df_bike_type_agg['bikes_out_classic_bike'] + df_bike_type_agg['bikes_in_classic_bike'])/2
df_bike_type_agg['total_electric_bikes'] = (df_bike_type_agg['bikes_out_electric_bike'] + df_bike_type_agg['bikes_in_electric_bike'])/2

plt.figure(figsize=(12, 6))
plt.bar(df_bike_type_agg['date_only'], df_bike_type_agg['total_classic_bikes'], label='Classic Bikes')
plt.bar(df_bike_type_agg['date_only'], df_bike_type_agg['total_electric_bikes'], bottom=df_bike_type_agg['total_classic_bikes'], label='Electric Bikes')

plt.title('Bike Type Usage Over Time')
plt.xlabel('Date')
plt.ylabel('Total Usage')
plt.legend()
plt.show()